# default segment code

In [18]:
from zhsegment import *

class Segment:

    def __init__(self, Pw):
        self.Pw = Pw

    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []
        segmentation = [ w for w in text ] # segment each char into a word
        return segmentation

    def Pwords(self, words): 
        "The Naive Bayes probability of a sequence of words."
        return product(self.Pw(w) for w in words)

## Run the default solution on dev

In [19]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    
    def __call__(self, key):
        if key in self: 
            return self[key]/self.N
        else: 
            #print("Missing")
            #if len(key) == 1:
            return self.missingfn(key, self.N)
            #else:
            #    return -1000000
        
def punishLong(word, N):

    "Estimate the prob of unknown word while accounting for word length"
    return 10. / (N * 50**(len(word)))



In [20]:
Pw = Pdist(data=datafile("../data/count_1w.txt"))
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt", encoding='utf-8') as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签 订 高 科 技 合 作 协 议
新 华 社 上 海 八 月 三 十 一 日 电 （ 记 者 白 国 良 、 夏 儒 阁 ）
“ 中 美 合 作 高 科 技 项 目 签 字 仪 式 ” 今 天 在 上 海 举 行 。


## Evaluate the default output

In [21]:
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r', encoding='utf-8') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)


score: 0.27


In [22]:
# Segment from the previous assignment

class Segment:

    def __init__(self, Pw):
        self.Pw = Pw

    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []
        candidates = ([first]+self.segment(rem) for first,rem in self.splits(text))
        return max(candidates, key=self.Pwords)

    def splits(self, text, L=3):
        "Return a list of all possible (first, rem) pairs, len(first)<=L."
        return [(text[:i+1], text[i+1:])
                for i in range(min(len(text), L))]

    def Pwords(self, words):
        "The Naive Bayes probability of a sequence of words."
        return product(self.Pw(w) for w in words)


In [23]:
Pw = Pdist(data=datafile("../data/count_1w.txt"))
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
#output_full = []
with open("../data/input/dev.txt", encoding='utf-8') as f:
    text = f.readlines()
    refLine = text[0]
    # Check out text
    print(text[0])
    # Split it the first part and evaluate
    print(max(segmenter.splits(refLine), key=segmenter.Pwords))
   

中美在沪签订高科技合作协议

('中', '美在沪签订高科技合作协议\n')


# Create the heapq 

In [24]:
import heapq

class Entry(object):
    
    def __init__(self, word, startPos, logProb, backPoint):
        super(Entry, self).__init__()
        self.word = word
        self.startPos = startPos
        self.logProb = logProb
        self.backPoint = backPoint
        
    def __eq__(self, other):
        if not isinstance(other, type(self)): return NotImplemented
        return (self.startPos == other.startPos) and (self.word == other.word) and (self.logProb == other.logProb)

    def __lt__(self, other):
        if not isinstance(other, type(self)): return NotImplemented
        #if self.startPos == other.startPos:
        return self.logProb > other.logProb
        #else:
        #    return self.startPos < other.startPos
    
    def __gt__(self, other):
        if not isinstance(other, type(self)): return NotImplemented
        return self.logProb <= other.logProb
        #return self.startPos > other.startPos
    
    def print(self):
        
        print("Word: {} Start: {} End: {} Log Prob: {} Back: {}".format(self.word, self.startPos, self.startPos + len(self.word), self.logProb, self.backPoint))
    

Test the heapq

In [25]:
import math
# To get some word count it seems: segmenter.Pw[firstW]
h = []
chart = {}
firstW = segmenter.splits(refLine)[0][0]
print(firstW)
print(segmenter.Pw.N)
print(math.log(segmenter.Pw[firstW] / segmenter.Pw.N))
first = Entry(firstW, 0, math.log(segmenter.Pw[firstW] / segmenter.Pw.N), None)
heapq.heappush(h, first)
first.print()
#print(len(h))
#print(heapq.heappop(h))

中
85105.0
-5.637908261732704
Word: 中 Start: 0 End: 1 Log Prob: -5.637908261732704 Back: None


In [26]:
# Gather all possible inputs at position 0
h = [Entry(s[0], 0, math.log(segmenter.Pw(s[0])), None) for s in segmenter.splits(refLine)]
#h = [Entry(s[0], 0, math.log(segmenter.Pw[s[0]] / segmenter.Pw.N), None) for s in segmenter.splits(refLine)]
for entry in h:
    entry.print()
    
he = []
for s in h:
    heapq.heappush(he, s)
#print(he)
#print(len(he))
#print(len(refLine))

Word: 中 Start: 0 End: 1 Log Prob: -5.637908261732704 Back: None
Word: 中美 Start: 0 End: 2 Log Prob: -11.351641067242074 Back: None
Word: 中美在 Start: 0 End: 3 Log Prob: -11.351641067242074 Back: None


In [27]:
with open("../data/input/dev.txt", encoding='utf-8') as f:
    text = f.readlines()
    refLine = text[0]

Pw = Pdist(data=datafile("../data/count_1w.txt"), missingfn=punishLong)
segmenter = Segment(Pw)
for s in segmenter.splits(refLine):
    print(s[0])
    print(Pw(s[0]))


中
0.0035603078550026437
中美
4.700076376241114e-08
中美在
9.400152752482228e-10


# Test if the pipeline is correct

In [28]:
with open("../data/input/dev.txt",encoding='utf-8') as f:
    text = f.readlines()
    refLine = text[0]

L = 6
Pw = Pdist(data=datafile("../data/count_1w.txt"), missingfn=punishLong)
segmenter = Segment(Pw)

# Gather all possible inputs at position 0
chart = {}
refLine = refLine.strip()
print(refLine)

h = []

#########################################################
## initialize the heapq by adding all words at index 0 ##
#########################################################
for s in segmenter.splits(refLine, L):
    if s[0] in segmenter.Pw:
        h.append(Entry(s[0], 0, math.log(segmenter.Pw(s[0])), None))
        print("adding existing word: " + s[0])
        
print("\n")       

# push words into heapq
he = []
for s in h:
    heapq.heappush(he, s)

# initialize chart
for i in range(len(refLine)):
    chart[i] = Entry(None, None, None, None)
    
endDex = 0

##################################################################
## interative loop, find best entry from heapy and update chart ##
##################################################################
while len(he) > 0:
    # pop best entry (might reconsider older entry)
    print("Popping")
    entry = heapq.heappop(he)
    endDex = entry.startPos + len(entry.word)  - 1  
    print("Current endDex: {}".format(endDex)) # idx of best entry
    print("Pop Current Entry")
    entry.print()
    
    print("\n")  
    
    # update chart to current best entry
    if chart[endDex].backPoint is not None:
        prevEntry = chart[endDex].backPoint
        # Reverse as log probs
        if entry.logProb > prevEntry.logProb:
            print("changing entry ptr")
            chart[endDex] = entry
        if entry.logProb <= prevEntry.logProb:
            print("Continuing, no change in entry ptr")
            continue
    else:
        chart[endDex] = entry
        print("add entry for null ptr")
        
    print("\n") 
    
    h = []
    threshold = 1
    #####################################################
    ## update log prob according to the new best entry ##
    #####################################################
    for s in segmenter.splits(refLine[endDex +1:], L):
        if s[0] in segmenter.Pw:
            print("add in "+s[0])
            h.append( ( Entry(s[0], endDex + 1, entry.logProb + math.log(segmenter.Pw(s[0])), entry), "Pres") )
        elif len(s[0]) <= threshold:
            print("add in unseen "+s[0])
            h.append( ( Entry(s[0], endDex + 1, entry.logProb + math.log(segmenter.Pw(s[0])), entry), "Miss") )
    
    print("\n") 
    for s in h:
        #print("Possible Add")
        #s.print()
        if s[0] not in he:
            print("Adding")
            print(s)
            s[0].print()
            heapq.heappush(he, s[0])
    print("\n") 
            
    for k, v in chart.items():
        if v.word != None:
            print("Chart Entry: {}".format(k))
            v.print()
    
    print()
    #print(segmenter.splits(refLine[endDex + 1:]))
    #print("Check Lens Start: {} End: {}".format(entry.startPos, endDex))

中美在沪签订高科技合作协议
adding existing word: 中


Popping
Current endDex: 0
Pop Current Entry
Word: 中 Start: 0 End: 1 Log Prob: -5.637908261732704 Back: None


add entry for null ptr


add in 美


Adding
(<__main__.Entry object at 0x7f4ba67e4f98>, 'Pres')
Word: 美 Start: 1 End: 2 Log Prob: -13.325987682845131 Back: <__main__.Entry object at 0x7f4bb69e5358>


Chart Entry: 0
Word: 中 Start: 0 End: 1 Log Prob: -5.637908261732704 Back: None

Popping
Current endDex: 1
Pop Current Entry
Word: 美 Start: 1 End: 2 Log Prob: -13.325987682845131 Back: <__main__.Entry object at 0x7f4bb69e5358>


add entry for null ptr


add in 在


Adding
(<__main__.Entry object at 0x7f4ba67e4748>, 'Pres')
Word: 在 Start: 2 End: 3 Log Prob: -17.825386181035327 Back: <__main__.Entry object at 0x7f4ba67e4f98>


Chart Entry: 0
Word: 中 Start: 0 End: 1 Log Prob: -5.637908261732704 Back: None
Chart Entry: 1
Word: 美 Start: 1 End: 2 Log Prob: -13.325987682845131 Back: <__main__.Entry object at 0x7f4bb69e5358>

Popping
Current endDex: 2
P

# Implementation of unigram segmentator

In [29]:
################################
## implementation of unigram ##
################################
def segChinese(refLine, segmenter, L):
    
    # Gather all possible inputs at position 0
    chart = {}
    heap_q = []
    threshold = 3
    #########################################################
    ## initialize the heapq by adding all words at index 0 ##
    #########################################################
    # push words into heapq
    for s in segmenter.splits(refLine, L):
        if s[0] in segmenter.Pw or len(s[0]) <= threshold:
            heapq.heappush(heap_q, Entry(s[0], 0, math.log(segmenter.Pw(s[0])), None))

    # initialize chart
    for i in range(len(refLine)):
        chart[i] = Entry(None, None, None, None)
    
    endIdx = 0
    
    
    ##################################################################
    ## interative loop, find best entry from heapy and update chart ##
    ##################################################################
    while len(heap_q) > 0:
        # pop best entry (might reconsider older entry)
        entry = heapq.heappop(heap_q)
        endIdx = entry.startPos + len(entry.word)  - 1  
    
        # update chart to current best entry
        if chart[endIdx].backPoint is not None:
            prevEntry = chart[endIdx].backPoint
            if entry.logProb > prevEntry.logProb:
                chart[endIdx] = entry
            if entry.logProb <= prevEntry.logProb:
                continue
        else:
            chart[endIdx] = entry
            
        #####################################################
        ## update log prob according to the new best entry ##
        #####################################################
        h = []
        for s in segmenter.splits(refLine[endIdx +1:], L):
            if s[0] in segmenter.Pw or len(s[0]) <= threshold:
                h.append(Entry(s[0], endIdx + 1, entry.logProb + math.log(segmenter.Pw(s[0])), entry)) 
        
        # only add different entry
        for s in h:
            if s not in heap_q:
                heapq.heappush(heap_q, s)

       
    cWord = chart[len(chart) - 1]
    cWord
    res = []
    while cWord.backPoint != None:
        res.append(cWord.word)
        cWord = cWord.backPoint
    res.append(cWord.word)

    setSeg = res[::-1]
    
    return setSeg


# Got a score of 0.91 for the unigram

In [30]:
L = 6
Pw_uni = Pdist(data=datafile("../data/count_1w.txt"), missingfn=punishLong)
segmenter_uni = Segment(Pw_uni)


output_full = []
with open("../data/input/dev.txt", encoding='utf-8') as f:
    for line in f:
        tmpLine = line.strip()
        out = segChinese(tmpLine, segmenter_uni, L)
        output = " ".join(out)
        output_full.append(output)

    
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r', encoding='utf-8') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

score: 0.91


# Print out some test results using unigram

In [31]:
L = 6
Pw_uni = Pdist(data=datafile("../data/count_1w.txt"), missingfn=punishLong)
segmenter_uni = Segment(Pw_uni)


output_full = []
with open("../data/input/test.txt", encoding='utf-8') as f:
    for line in f:
        tmpLine = line.strip()
        out = segChinese(tmpLine, segmenter_uni, L)
        output = " ".join(out)
        output_full.append(output)

for line in output_full:
    print(line)
    


法 正 研究 从 波黑 撤军 计划
新华社 巴黎 ９月 １日 电 （ 记者 张有浩 ）
法国 国防部 长莱奥 塔尔 １日 说 ， 法国 正在 研究 从 波黑 撤军 的 计划 。
莱奥塔 尔是在 巴黎 向 一家 电 台 记者 发表 谈话 时 说 这番话 的 。
他 指出 ， 美国 建议 解除 对 波黑 的 武器 禁运 ， 意味 着 “ 国际 援助 可能 结束 ” 以及 “ 重大 的 公开 冲突 再次 爆发 ” ， 而 这 将 产生 “ 造成 重大 损失 的 严重 后果” 。
莱奥 塔尔说 ， “ 在 联合国 维和 部队 的 安全 得 不到 保证 的 情况 下 ， 我们 不 能 接受 解除 对 波黑 的 武器 禁运 ” 。
法国 外长 朱佩 上周也 明确 表示 ， 解除 禁运 的 “先决 条件 是 法国 撤走 其蓝盔 部队 以及 所有 联合国 维和 部队 的 撤离 ” 。
１９９２年 ２月 波黑 冲突 爆发 以来 ， 法国 在 波黑 一直 派驻 有 约 ４００ ０名官 兵 。
它 是 向 该 地区 派驻 蓝盔 部队 最 多 的 西方 国家 。
美国 总统 克林顿 ８月 １０日 宣称 ， 如果 波黑 塞族 到 １０月 １５日 仍 不 接受 ５ 国 和平 方案 ， 他 将 请求 安理会 通过 决议 ， 取消 对 波黑 的 武器 禁运 ； 如果 安理会 不 能 通过 这样 的 决议 ， 他 将 谋求 美国 单方面 解除 对 波黑 的 武器 禁运 。
（ 完 ）
刘华清 会见 泰国 副总理
新华社 北京 九月 一日 电 （ 记者 杨国钧 ）
中央 军委 副主席 刘华清 今天 在 钓鱼台 国宾馆 会见 泰国 副总理 兼 内政 部长 差瓦立 ·永猜 裕上将 时 说 ， 中国 十分 重视 发展 同 泰国 政府 、 人民 和 军队 之间 的 友谊 ， 特别是 两 国 经贸 之间 的 合作 。
刘华清 说 ， 中 泰两国 人民 有 传统 的 友谊 ， 两 国 的 关系 也 十分 友好 。
他 说 ， 差瓦立 副总理 的 这 次 来访 以及 中 泰两国 其他 高层 领导人 的 互访 ， 有利于 推进 两 国 关系 的 进一步 发展 以及 在 各 领域 的 有效 合作 。
差瓦立 说 ， 他 十分 高兴 会见刘 副主席 。
他 说 ， 近年 来 ， 中国 经济 发展 迅速 ， 泰国 政府 和

# Now let's implement bigram + unigram + smoothing

In [32]:
############################################################
## implementation of bi-gram and uni-gram with smoothing ###
############################################################
def segChinese(refLine, segmenter_uni, segmenter_bi, L):
    
    # Gather all possible inputs at position 0
    chart = {}
    heap_q = []
    threshold = 3
    #########################################################
    ## initialize the heapq by adding all words at index 0 ##
    #########################################################
    # push words into heapq
    for s in segmenter_uni.splits(refLine, L):
        if s[0] in segmenter_uni.Pw or len(s[0]) <= threshold:  # use uni-gram segmenter for splitting
            heapq.heappush(heap_q, Entry(s[0], 0, math.log(segmenter_uni.Pw(s[0])), None))
           
    # initialize chart
    for i in range(len(refLine)):
        chart[i] = Entry(None, None, None, None)
        
    endIdx = 0
    
    ##################################################################
    ## interative loop, find best entry from heapy and update chart ##
    ##################################################################
    while len(heap_q) > 0:
        # pop best entry (might reconsider older entry)
        entry = heapq.heappop(heap_q)
        endIdx = entry.startPos + len(entry.word)  - 1  
    
        # update chart to current best entry
        if chart[endIdx].backPoint is not None:
            prevEntry = chart[endIdx].backPoint
            if entry.logProb > prevEntry.logProb:
                chart[endIdx] = entry
            if entry.logProb <= prevEntry.logProb:
                continue
        else:
            chart[endIdx] = entry
            
        #####################################################
        ## update log prob according to the new best entry ##
        #####################################################
        h = []
        for s in segmenter_uni.splits(refLine[endIdx +1:], L):
            if s[0] in segmenter_uni.Pw or len(s[0]) <= threshold:
                bi_gram_word = entry.word +" " +s[0]
                uni_gram_prob = segmenter_uni.Pw(s[0])
                
                if bi_gram_word in segmenter_bi.Pw: # use bi-gram if possible
                    assert entry.word in segmenter_uni.Pw # assert first word appears in 1w dictionary
                    bi_gram_prob = (segmenter_bi.Pw(bi_gram_word)*Pw_bi.N) / (segmenter_uni.Pw(entry.word)*Pw_uni.N)
                    h.append(Entry(s[0], endIdx + 1, entry.logProb + math.log(0.1*bi_gram_prob+0.9*uni_gram_prob), entry)) 
                    
                else: # else bi-gram has probability zero, smoothing with unigram
                    h.append(Entry(s[0], endIdx + 1, entry.logProb + math.log(0.1*0.0+0.9*uni_gram_prob), entry)) 
        
        for s in h: # only add different entry
            if s not in heap_q:
                heapq.heappush(heap_q, s)

       
    cWord = chart[len(chart) - 1]
    cWord
    res = []
    while cWord.backPoint != None:
        res.append(cWord.word)
        cWord = cWord.backPoint
    res.append(cWord.word)

    setSeg = res[::-1]
    
    return setSeg


# Score is now 0.92

In [33]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./N)
    
    def __call__(self, key):
        if key in self: 
            return self[key]/self.N
        else: 
            #print("Missing")
            #if len(key) == 1:
            return self.missingfn(key, self.N)
            #else:
            #    return -1000000
        
def punishLong(word, N):

    "Estimate the prob of unknown word while accounting for word length"
    return 10. / (N * 8000**(len(word)))


L = 6
Pw_uni = Pdist(data=datafile("../data/count_1w.txt"), missingfn=punishLong)
Pw_bi = Pdist(data=datafile("../data/count_2w.txt"), missingfn=punishLong)
segmenter_uni = Segment(Pw_uni)
segmenter_bi = Segment(Pw_bi)


output_full = []
with open("../data/input/dev.txt", encoding='utf-8') as f:
    for line in f:
        tmpLine = line.strip()
        out = segChinese(tmpLine, segmenter_uni, segmenter_bi, L)
        output = " ".join(out)
        output_full.append(output)

#for line in output_full:
    #print(line)
    
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r', encoding='utf-8') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)


score: 0.92


# Let us print out some test results

In [17]:
with open("../data/input/dev.txt", encoding='utf-8') as f:
    for line in f:
        tmpLine = line.strip()
        out = segChinese(tmpLine, segmenter_uni, segmenter_bi, L)
        output = " ".join(out)
        output_full.append(output)

for line in output_full:
    print(line)
    

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。
上午 在 这里 签字 的 是 知识 信息 网络 通讯 技术 和 脱氧核 糖核酸 生物 技术 两 个 项目 ， 同时 还 签订 了 语言 教学 交流 合作 协议 。
这 三 个 项目 是 分别 由 国务院 发展 研究 中心 国际 技术 经济 研究所 上海 分 所 和 上海市 浦东 继续 教育 中心 ， 与 美国 知识 信息 网络 公司 、 世界 学习 组织 、 海赛克 公司 签订 的 。
知识 信息 网络 公司 主席 ＣＥ 麦奈特 在 签字 仪式 上 说 ， 中 美 合作 高 科技 项目 的 签订 ， 具有 重要 的 意义 。
他 还 期望 美 中 之间 建立 电子 高速 信息 公路 ， 使 纽约 股票 交易所 与 上海 证券 交易所 能 早日 实现 交易 及 信息 交流 。
世界 学习 组织 董事长 朱迪 ·梅罗 希望 在 今年 底 与 上海 建立 一 个 合资 的 教育 机构 。
据 美方 人员 说 ， 今天 签订 的 三 个 项目 ， 得到 前来 上海 的 美国 商务 部长 布朗 的 关心 与 支持 。
（ 完 ）
中国 将 于 十二月 举办 “ 华 夏 艺术 节 ”
新华社 北京 九月 一日 电 （ 记者 高 建 新 ）
为 加强 海内外 炎黄子孙 的 文化 交流 和 联系 ， 中国 将 于 今年 十二月 八日 到 十八日 在 北京 和 深圳 两 地 同时 举办 “ 华 夏 艺术 节 ” 。
这 是 一 次 由 人民 团体 、 政府 文化 部门 和 企业 联手 举办 的 艺术 盛会 。
其 宗旨 是 ， 以 中华 民族 传统 文化 为 纽带 ， 联系 和 团结 海内外 同胞 ， 弘扬 民族 文化 ， 展示 祖国 优秀 艺术 的 魅力 ， 展示 华侨 、 华人 艺术 家 在 艺术 领域 和 世界 文化 交流 中 的 成就 和 风采 。
艺术 节 的 内容 主要 包括 隆重 的 开幕式 和 闭幕 式 ， 各 具 特色 的 大型 文艺 表演 以及 美 术 、 摄影 展览 。
据 介绍 ， 十二月 八日 在 深圳 体育馆 开幕 的 艺术 节 ， 将 集中 展示 二十 世纪 广为 

# Now let's manually compare the two results

Example 1:
    Bi-gram + Uni-gram + Smoothing (BUS): 法 正 研究 从...   
    Uni-gram (UG): 法正 研究 从
    Here BUS is correct, UG is wrong. The correct interpolation should be France is considering...
    法 refers to France and 正 refers to is. There are two words but UG thinks it is one word.
    
Example 2:
    BUS: 他 将 请求
    UG: 他 将请求
    Again BUS is correct but UG is wrong. The correct interpolation is He(他) will(将) demand(请求)...
    UG thinks will and demand are a single word.
    
Example 3:
    BUS: 他 十分 高兴 会见 刘 副主席
    UG: 他 十分 高兴 会见刘 副主席
    BUS is correct, UG is incorrect. Ground truth is 他(He) 十分 高兴 (is very happy) 会见 (to see) 刘 (Mr. Liu) 副主席 (vice-chairman).
    UG thinks 会见(to see) and 刘(Mr. Liu) are one word.
    
There are many more examples. Overall, we found that bi-gram + uni-gram + smoothing works much better than simply using uni-gram.
    